In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import math
%matplotlib inline

pd.set_option('display.max_columns', 120)

In [46]:
# A custom function to compute the R score
def get_reward(y_true, y_fit):
    R2 = 1 - np.sum((y_true - y_fit)**2) / np.sum((y_true - np.mean(y_true))**2)
    R = np.sign(R2) * math.sqrt(abs(R2))
    return(R)

In [2]:
with pd.HDFStore("./train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

In [39]:
d_median = df.median(axis=0)
n = df.isnull().sum(axis=1)
for c in df.columns:
    df[c + '_nan_'] = pd.isnull(df[c])
    d_median[c + '_nan_'] = 0
df = df.fillna(d_median)
df['znull'] = n
n = []

In [15]:
train = df[:806298]
val = df[806298:]

In [24]:
train_labels = train["y"]
val_labels = val["y"]
train_data = train.drop("y", 1)
val_data = val.drop("y", 1)

In [32]:
gbm = xgb.XGBRegressor(nthread=7, silent=False, n_estimators=300, learning_rate=.2)

In [33]:
gbm.fit(train_data, train_labels)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.2, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=300, nthread=7,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [40]:
predictions = gbm.predict(train_data)

In [47]:
print get_reward(predictions, train_labels)

-12.8572098866


In [41]:
print predictions[:50]

[  2.28205323e-03   2.20924616e-04  -6.90340996e-04  -4.36484814e-04
  -8.96215439e-04  -2.88927555e-03  -1.28966570e-03   1.14968419e-03
  -1.71458721e-03  -2.89607048e-03   3.89546156e-04  -1.47461891e-04
   2.20924616e-04  -1.42294168e-03   2.20924616e-04  -2.38996744e-03
  -6.77204132e-03  -2.09033489e-04  -3.52543592e-03   6.01619482e-04
   1.37555599e-03   1.29643083e-03   1.26177073e-03   3.49074602e-04
  -3.23724747e-03  -1.60336494e-04   6.50525093e-04  -3.46374512e-03
   1.83847845e-02  -9.81092453e-05   1.38613582e-03  -2.08004713e-02
  -7.42554665e-04   1.86413527e-04  -1.14046931e-02   4.69297171e-04
  -2.88665295e-03  -8.87292624e-03  -7.55250454e-04   1.29938126e-04
  -5.25808334e-03   1.99380517e-03   2.02363729e-03   1.12435222e-03
  -1.96284056e-03  -2.57462263e-03   1.70806646e-02   1.26177073e-03
  -7.13145733e-03  -9.85860825e-05]


In [42]:
print train_labels[:50]

0    -0.011753
1    -0.001240
2    -0.020940
3    -0.015959
4    -0.007338
5     0.031425
6    -0.032895
7     0.015803
8    -0.027593
9     0.006662
10   -0.001899
11    0.050219
12   -0.018991
13   -0.005203
14   -0.006369
15    0.017768
16   -0.001089
17   -0.008794
18    0.040724
19   -0.003921
20   -0.011317
21    0.044167
22    0.001395
23   -0.012101
24   -0.070837
25   -0.001766
26   -0.012542
27    0.029697
28    0.030815
29   -0.016336
30    0.041231
31   -0.086094
32   -0.019769
33   -0.019097
34   -0.042154
35    0.015426
36   -0.026304
37   -0.015709
38   -0.002253
39   -0.011964
40   -0.041490
41    0.016232
42   -0.020027
43   -0.003844
44   -0.024898
45   -0.013219
46    0.066793
47   -0.002048
48    0.000455
49   -0.023536
Name: y, dtype: float32
